# Single instance

In [ ]:
using Yao, QAOA, Random, Combinatorics, Arpack, Distributions, LinearAlgebra, Interpolations
using JLD, HDF5, Printf
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")

PATH = "/home/ubuntu/Archives/";

In [ ]:
using Revise, SpinFluctuations

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)

In [ ]:
N = 9
pattern = r"random_SK_instance_N_9_seed_(\d+)\.h5"

subdir = "small_gaps"
# subdir = "large_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name);

In [ ]:
T_final = 32000.

# number of points to get Lyapunov exponent for
npts = 256
# npts = 2048

# tolerance for DifferentialEquations.jl when solving mean-field 
tol = 1e-8;

In [ ]:
# N = 9
seed = 6
seed = 42
# seed = 4147
# seed = "6150"
# seed = "11068"
seed = 75
seed = 1004

# N = 15
# seed = "36"
# seed = "325"
# seed = "19015"
# seed = "19134"
# seed = "20484"
J_mat = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "J")
λ = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "exact_ARPACK_LM_eigvals");

# seed = "2015"
# J_mat = couplings_large_gap[seed];
# λ = eigvals_large_gap[seed];

In [ ]:
mf_problem = Problem(0, J_mat);

## Exact gap

In [ ]:
exact_times = range(0, 1, 33);

In [ ]:
gap = λ[2, :] .- λ[1, :];
mingap = minimum(gap) 
mingap |> println
gaploc = exact_times[findfirst(x -> x == minimum(gap), gap)] 
gaploc |> println

In [ ]:
small_idxs = findall(x -> x < 0.15, gap) 
small_idxs |> println

gap_idx = findfirst(x -> x == mingap, gap) 
gap_idx |> println
small_idxs[1:findfirst(x -> x == gap_idx, small_idxs) + 1] |> println
exact_times[small_idxs[1:findfirst(x -> x == gap_idx, small_idxs) + 1]] |> println

In [ ]:
figure(figsize=(4, 3))
subplot(111)
for i in 1:size(λ)[1]
    plot(exact_times, (λ[i, :] .- λ[1, :]), "-k", ms=2)
end
axvline(exact_times[small_idxs[1]], c="r")
axvline(exact_times[gap_idx], c="r")
xlim(0., 1.)
ylim(0, 2)
xlabel("\$s\$")
ylabel("Exact Eigenvalues")
tight_layout()

In [ ]:
bogo_spec = bogoliubov_spectrum(mf_problem, LyapunovParameters(32000, 32, tol, tol))
bogo_spec = reduce(hcat, bogo_spec)
bogo_spec = sort(bogo_spec .|> real, dims=1);

In [ ]:
figure(figsize=(4, 3))
subplot(111)
for i in 1:size(bogo_spec)[1]
    # plot(exact_times[2:end], bogo_spec[i, :], "-k", ms=2)
    # plot(exact_times, (λ[i, :] .- λ[1, :]), "--r", ms=2)
    plot(exact_times[2:end], bogo_spec[i, :] ./ 2pi, "--r", ms=2)
    plot(exact_times, (λ[i, :] .- λ[1, :]) ./ 2pi, "-k", ms=2)    
end
xlim(0., 1.)
ylim(-0., 0.5)
xlabel("\$s\$")
ylabel("Exact Eigenvalues")
tight_layout()

## Mean-field trajectories

In [ ]:
# T_final = 1000. 
# T_final = 4000. 
# T_final = 8000. 
# T_final = 16000.
T_final = 32000.
# T_final = 64000.
tol = 1e-8;

In [ ]:
sol = evolve_mean_field(mf_problem.local_fields, mf_problem.couplings, T_final, t -> t/T_final, rtol=1e2*tol, atol=tol)
size(sol.t)[1] 

In [ ]:
component_dict = Dict("x" => 1, "y" => 2, "z" => 3)
component = "z"
test_data = reduce(hcat, [sol.u[k][component_dict[component], :] for k in 1:size(sol.u)[1]]);

In [ ]:
figure(figsize=(4, 3))

subplot(111)
for spin_nr in 1:N-1
    if round(abs.(test_data[spin_nr, :][end]), digits=3) == 1.0
        plot(sol.t ./ T_final, test_data[spin_nr, :], "-")
    else
        plot(sol.t ./ T_final, test_data[spin_nr, :], "-", label=@sprintf("\$i=%i\$", spin_nr))    
    end
end

xlim(0, 1)
ylim(-1, 1)
xlabel("\$s\$")
ylabel("\$n_i^z(s)\$")
legend(frameon=false)

tight_layout()
# savefig("../plots/" * @sprintf("mean_field_max2sat_typical_instance_%04i_from_arxiv_2206_06876_N_%i_num_clauses_%i.pdf", idx, N, num_clauses), dpi=256, bbox_inches="tight")

In [ ]:
sigma_star = sign.(sol.u[end][3, :])
h = mf_problem.local_fields
J = mf_problem.couplings
E = sum([-h[l] * sigma_star[l] for l in 1:N-1]) + sum([-J[i, j] * sigma_star[i] * sigma_star[j] for i in 1:N-1 for j in (i+1):N-1])

In [ ]:
λ[1, :][end]

## Statistical Green function

In [ ]:
tol = 1e-8
# T_final = 32000.
# npts = 2048
coarse_times = range(0, 1, npts + 1);

In [ ]:
lyapunov_parameters = LyapunovParameters(T_final, npts, tol, tol)
mf_sol, stat_GF = statistical_green_function(mf_problem, lyapunov_parameters);

In [ ]:
flucs = k -> (real.(1.0im .* diag(stat_GF[k])[1:mf_problem.num_qubits]) .- 1.0) ./ 2;
all_flucs = reduce(hcat, map(flucs, 1:npts+1));

In [ ]:
figure(figsize=(20, 4))
ylims = 0.25
for i in 1:(N-1)÷2
    ax = subplot(2, (N-1)÷2, i)
    plot(coarse_times, all_flucs[i, :], label=@sprintf("\$i=%s\$", string(i)))
    # axvline(coarse_times[findfirst(x -> x == maximum(all_flucs[i, :]), all_flucs[i, :])], c="r")
    xlim(0., 1.)
    ax.set_xticklabels([])
    ylim(0., ylims)
    if i > 1
        ax.set_yticklabels([])
    end
    legend(frameon=false, ncol=2, handlelength=0)
end

ax = subplot(2, (N-1)÷2, 1)
ax.set_ylabel("\$F_{ii}(t, t)\$")

for i in (N-1)÷2+1:N-1
    ax = subplot(2, (N-1)÷2, i)
    plot(coarse_times, all_flucs[i, :], label=@sprintf("\$i=%s\$", string(i)))
#     println(i, ": ", maximum(all_flucs[i, :]))
    # axvline(coarse_times[findfirst(x -> x == maximum(all_flucs[i, :]), all_flucs[i, :])], c="r")    
    xlim(0., 1.)
    ylim(0., ylims)
    if i > (N-1)÷2+1
        ax.set_yticklabels([])
    end    
    xlabel("\$s\$")    
    legend(frameon=false, ncol=2, handlelength=0)
end

ax = subplot(2, (N-1)÷2, (N-1)÷2 + 1)
ax.set_ylabel("\$F_{ii}(t, t)\$")

tight_layout()
# savefig("../plots/" * @sprintf("fluctuations_max2sat_typical_instance_%04i_from_arxiv_2206_06876_N_%i_num_clauses_%i.pdf", idx, N, num_clauses), dpi=256, bbox_inches="tight")

## Spectra

In [ ]:
# npts_diag = 16
# τ_final = 1500.
# spectral_sols = evolve_spectral_function(mf_problem, T_final, τ_final, npts_diag=npts_diag, rtol=1e-6, atol=1e-6);

In [ ]:
npts_diag = 16
T_diags = T_final .* range(0.2, 0.8, npts_diag+1)
# T_diags = T_final .* exact_times[small_idxs[1:findfirst(x -> x == gap_idx, small_idxs) + 1]] 
τ_final = 2000.;

In [ ]:
_, spectral_sols = evolve_spectral_function(mf_problem, T_final, τ_final, T_diags, rtol=1e-6, atol=1e-6);

spec_sum = []
for k in 1:length(T_diags)
    push!(spec_sum, spectral_fft(spectral_sum(spectral_sols[k])))
end

In [ ]:
_, spectral_sums = evolve_spectral_sum(mf_problem, T_final, τ_final, T_diags, rtol=1e-4, atol=1e-6);

spec_sum = []
for k in 1:length(T_diags)
    push!(spec_sum, spectral_fft(spectral_sums[k]))
end

In [ ]:
spectral_data = Dict();

In [ ]:
for spin_idx in 1:N-1
    spectral_data[spin_idx] = spectrum_vs_T(spin_idx, spectral_sols)
end

In [ ]:
spin_idx = 4

# map data to functions
spectral_fs = []
for (k, (ωs, spec)) in enumerate(zip(spectral_data[spin_idx]...))
    push!(spectral_fs, linear_interpolation(ωs, spec, extrapolation_bc=Line()))
end

In [ ]:
# spin_idx = 2
num_plots = length(spectral_sols)
xlim_vals = (-0.05, 0.3)
# fig, axs = subplots(figsize=(5, 12), nrows=num_plots, ncols=1)
fig, axs = subplots(figsize=(5, 8), nrows=num_plots, ncols=1)

# T_diags = range(0, 1, npts_diag+1)
for (k, (ωs, spec)) in enumerate(zip(spectral_data[spin_idx]...))
    ax = axs[k] # subplot(num_plots, 1, k)     

    ax.plot(spec_sum[k]..., "-C2")

    ax.plot(ωs, spec, "-", label=@sprintf("\$T=%0.3f\$", T_diags[k] ./ T_final))
    # ax.axvline(mingap, ls="--", c="k", alpha=0.2)
    # max_loc = ωs[findfirst(x->x==maximum(spec), spec)]
    # axvline.([max_loc], c="r", ls="--")

    ax.set_xlim(xlim_vals...)
    ax.set_ylim(0, 0.1)  
    ax.set_xticks(range(xlim_vals..., 8))  
    ax.set_xticklabels([])
    # ax.set_ylabel(@sprintf("\$\\rho_{%i%i}(T, \\omega)\$", idx, idx))
    ax.legend(frameon=false, handlelength=0, fontsize=12)
    if k%2 == 0
        ax.yaxis.tick_right()
    end
    ax.tick_params(axis="y", labelsize=10)
end
ax = subplot(num_plots, 1, num_plots)
ax.set_xlabel("\$\\nu\$")
ax.set_xticklabels(round.(range(xlim_vals..., 8), digits=2))

fig.text(0.0, 0.5, @sprintf("\$\\rho_{%i%i}(T, \\omega)\$", spin_idx, spin_idx), va="center", rotation="vertical")
subplots_adjust(wspace=0.5, hspace=0.16)
# savefig("../plots/" * @sprintf("spectrum_spin_%02i_max2sat_typical_instance_%04i_from_arxiv_2206_06876_N_%i_num_clauses_%i.pdf", spin_idx, idx, N, num_clauses), dpi=256, bbox_inches="tight")

In [ ]:
T_diags = T_final .* range(0.5, 1.0, npts_diag+1)
@sprintf("spectra_T_final_%i_tau_final_%i/T_%0.5f/omegas", T_final, τ_final, T_diags[4] / T_final)

In [ ]:
omega_range = range(-0.02, 0.3, 1001) |> collect
data_2D = [map(spectral_fs[k], omega_range) for (k, (ωs, spec)) in enumerate(zip(spectral_data[spin_idx]...))];

In [ ]:
cmap = "gist_yarg"

figure(figsize=(4, 3))
Y, X = meshgrid(omega_range, T_diags ./ T_final);
ax = subplot(111)
heatmap = ax.pcolormesh(Y, X, data_2D, cmap=cmap, rasterized=true, vmin=0, vmax=0.1)
heatmap.set_edgecolor("face")
# ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
# cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel("\$\\omega\$")
ax.set_ylabel("\$T\$")
ax.set_xlim(omega_range[1], omega_range[end])
ax.set_ylim(T_diags[1] ./ T_final, T_diags[end] ./ T_final)
# ax.set_xticks(t_scale .* [0, T/2, T])
# ax.set_yticks(t_scale .* [0, T/2, T])

tight_layout(pad=0.75, w_pad=0.25, h_pad=0)

In [ ]:
data_2D_f = interpolate((omega_range, T_diags), reduce(hcat, data_2D), Gridded(Linear()));
interp_data_2D = [[data_2D_f(ω, T) for ω in omega_range] for T in range(T_diags[1], T_diags[end], 1001)];

In [ ]:
figure(figsize=(4, 3))
Y, X = meshgrid(omega_range, range(T_diags[1], T_diags[end], 1001) ./ T_final);
ax = subplot(111)
heatmap = ax.pcolormesh(Y, X, interp_data_2D, cmap=cmap, rasterized=true, vmin=0, vmax=1)
heatmap.set_edgecolor("face")
# ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
ax.set_xlim(omega_range[1], omega_range[end])
# ax.set_ylim(0, 1)
# ax.set_xticks(t_scale .* [0, T/2, T])
# ax.set_yticks(t_scale .* [0, T/2, T])

tight_layout(pad=0.75, w_pad=0.25, h_pad=0)